#**MonomerFinder: Identificación de microplásticos**

In [1]:
#instalaciones requeridas
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install roboflow
!pip install folium

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-qmk0g4so
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-qmk0g4so
  Resolved https://github.com/facebookresearch/detectron2.git to commit 2a420edb307c9bdf640f036d3b196bed474b8593
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.6 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [2]:
!pip install --quiet gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━

##LIBRERÍAS

In [3]:
#Librerías análisis

#De uso general
import os #Para funcionalidades dependientes del sistema operativo
import cv2 #Para el procesamiento de imágenes
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt #Para crear gráficos
import numpy as np #Para operar funciones, vectores y matrices
import pandas as pd #Para manipulación y análisis de datos
from PIL import Image #Para abrir, manipular y guardar imágenes en varios formatos

#Librerías interfaz
import gradio as gr #Para el entorno
import io
import base64

#Conjunto de librerías para geolocalización
import folium #Para la creación de mapas interactivos
from IPython.display import display

%matplotlib inline
#Conjunto de librerías para la detección de objetos
import torch #ML basada en el cálculo automático de gradientes y en redes neuronales profundas.
import detectron2 #Detección de objetos.
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultTrainer
import logging #Para registrar eventos y mensajes durante la ejecución del programa
from detectron2.utils.logger import setup_logger
from roboflow import Roboflow #Ofrece datasets y herramientas para el entrenamiento de modelos


##MODELO DE CONTEO DE MICROPLÁSTICOS

In [4]:
#Descarga y configuración del dataset
rf = Roboflow(api_key="Dlwgwe3psTHRPIuQlsbV")
project = rf.workspace("panats-mp-project").project("microplastic-dataset")
dataset = project.version(19).download("coco")

#Definición de rutas y nombres
DATA_SET_NAME = dataset.name.replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

#Registro del dataset
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

#Validación del dataset
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

# Configuración del modelo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (VALID_DATA_SET_NAME,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 3000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7  # Número de categorías de microplásticos


# Creación del entrenador
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Configuración del modelo
cfg.MODEL.WEIGHTS = "./output/model_final.pth"  # Ruta a los pesos del modelo entrenado
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Umbral para la predicción
cfg.DATASETS.TEST = ("microplastic-dataset-valid", )

predictor = DefaultPredictor(cfg) #Predictor

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Microplastic-Dataset-19 in coco:: 100%|██████████| 787/787 [00:00<00:00, 1859.66it/s]


[07/26 14:42:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:01, 99.9MB/s]                          
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[07/26 14:42:58 d2.engine.train_loop]: Starting training from iteration 0


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)


[07/26 14:43:12 d2.utils.events]:  eta: 0:16:32  iter: 19  total_loss: 3.692  loss_cls: 1.836  loss_box_reg: 0.7383  loss_rpn_cls: 0.9584  loss_rpn_loc: 0.123    time: 0.3386  last_time: 0.2926  data_time: 0.0293  last_data_time: 0.0053   lr: 1.9981e-05  max_mem: 1840M
[07/26 14:43:21 d2.utils.events]:  eta: 0:15:41  iter: 39  total_loss: 2.901  loss_cls: 1.54  loss_box_reg: 0.8652  loss_rpn_cls: 0.2804  loss_rpn_loc: 0.08974    time: 0.3227  last_time: 0.3155  data_time: 0.0110  last_data_time: 0.0077   lr: 3.9961e-05  max_mem: 1840M
[07/26 14:43:27 d2.utils.events]:  eta: 0:15:20  iter: 59  total_loss: 2.123  loss_cls: 1.083  loss_box_reg: 0.8242  loss_rpn_cls: 0.1729  loss_rpn_loc: 0.09893    time: 0.3173  last_time: 0.3059  data_time: 0.0128  last_data_time: 0.0056   lr: 5.9941e-05  max_mem: 1840M
[07/26 14:43:33 d2.utils.events]:  eta: 0:15:10  iter: 79  total_loss: 1.623  loss_cls: 0.6782  loss_box_reg: 0.7912  loss_rpn_cls: 0.09122  loss_rpn_loc: 0.04064    time: 0.3144  last_ti

##MAPA

In [5]:
#Crear el mapa
def crear_mapa(latitud, longitud):
    # Crear un mapa con la ubicación ingresada
    mapa = folium.Map(location=[latitud, longitud], zoom_start=12)
    folium.Marker([latitud, longitud], tooltip="Ubicación").add_to(mapa)

    # Convertir el mapa a HTML
    mapa_html = mapa._repr_html_()
    return mapa_html

def procesar_datos(latitud, longitud, descripcion, imagen):
    mapa_html = crear_mapa(latitud, longitud)

    # Procesar la imagen (convertirla a formato base64 para mostrarla)
    if imagen is not None:
        imagen_pil = Image.open(imagen)
        imagen_buffer = io.BytesIO()
        imagen_pil.convert("RGB").save(imagen_buffer, format="JPEG")  # Guardar como JPEG
        imagen_base64 = base64.b64encode(imagen_buffer.getvalue()).decode()
        imagen_url = f"data:image/jpeg;base64,{imagen_base64}"
    else:
        imagen_url = None

    return mapa_html, imagen_url

##TEXTOS

In [16]:
# Contenido para definiciones
def contenido_mononerfinder():
    texto = (
        "MonomerFinder es una aplicación que facilita la identificación y clasificación de microplásticos presentes en imágenes capturadas por los propios usuarios. Para esto utiliza una IA que se encarga de analizar las imágenes y señalar la cantidad de microplásticos presentes y su tipo. Esta aplicación es el resultado del Reto 5 del Hackaton Coafina 2024: 'Microplásticos: un desafío ciudadano'."
    )
    imagen_path = "/content/Imagenes/RaceforWater_PeterCharaf_MicroplasticsAzores_(2).jpg"
    imagen = Image.open(imagen_path)
    return texto, imagen

def contenido_fuentes_microplasticos():
    texto = (
        "Los microplásticos pueden provenir de diversas fuentes, entre las que se destacan:",
        "-Textiles: El 70% de los textiles producidos hoy en día son de origen sintético y liberan grandes cantidades de microplásticos al medio ambiente. La liberación de fibras y sustancias sintéticas de la ropa se produce no solo a través del lavado y el uso, sino también durante la producción, el procesamiento y el transporte. Los geotextiles, que se utilizan comúnmente para sostener y reforzar las capas del suelo o como manto agrícola, también liberan microplásticos debido a la exposición a los rayos UV, daños físicos y un mantenimiento deficiente.",
        "-Cosméticos y productos de cuidado personal: Un gran número de productos de cuidado personal todavía contienen microplásticos añadidos intencionadamente (por ejemplo, escarcha, microperlas en exfoliantes faciales y corporales). Estos productos ingresan al sistema de alcantarillado y a las plantas de tratamiento de aguas residuales. Aquí no se eliminan por completo y, por lo tanto, ingresan al medio ambiente.",
        "-Pesca y acuicultura: Los microplásticos son liberados por: aguas grises de los buques que desembocan sin filtrar en el mar, la degradación de las líneas de pesca perdidas, pinturas y revestimientos marinos, residuos plásticos de un solo uso procedentes de la pesca y la acuicultura, y pérdida de contenedores marítimos con productos plásticos.",
        "-Agricultura: Diversas fuentes agrícolas contribuyen a la propagación de microplásticos en el suelo, por ejemplo, a través de: el uso generalizado de películas de cultivo, tuberías de riego, comprimidos nutritivos, recubrimientos de semillas, lodos de depuradora de plantas de tratamiento de aguas residuales utilizados como fertilizante del suelo. Dado que la mitad de todos los lodos de depuradora de europa acaban de nuevo en tierra, se trata de un problema generalizado.",
        "-Tráfico: La abrasión de los neumáticos, las marcas viales y los escombros de la carretera son fuentes de microplásticos ambientales. Se estima que solo la abrasión de los neumáticos genera más de 1,3 millones de toneladas de microplásticos en Europa cada año. El 'reciclaje' de neumáticos usados suele ser problemático, ya que los microplásticos se liberan directamente al medio ambiente a través de su uso en arrecifes artificiales, campos deportivos o patios escolares.",
        "-Procesamiento de plásticos: Los pellets son perlas de plástico nuevas o recicladas que se utilizan como materia prima en la fabricación de la mayoría de los productos plásticos. Sin embargo, las pérdidas de gránulos, escamas y polvo de plástico se producen a lo largo de toda la cadena de valor de la producción, el transporte y el reciclaje de plástico. Además, el agua del proceso y las aguas residuales de la industria del plástico pueden estar muy contaminadas con microplásticos.",
        "-Turismo: La eliminación inadecuada de los residuos de los turistas, los sistemas locales de gestión de residuos deficientes o inexistentes, la descarga incontrolada de aguas residuales de los cruceros y el consumo masivo de productos y envases de plástico de un solo uso en los países impulsados por el turismo, aumentan los aportes de microplásticos."
    )
    imagen_path = "/content/Imagenes/Pollution_on_Land.jpg"
    imagen = Image.open(imagen_path)
    return texto, imagen

def contenido_microplasticos():
    texto = (
        "Los microplásticos son diminutas partículas de plástico, generalmente con tamaños menores a 5 mm, que se han convertido en un problema ambiental global. Provienen de diversas fuentes, como la degradación de objetos plásticos más grandes, productos cosméticos y fibras sintéticas de la ropa. Estos diminutos fragmentos contaminan océanos, ríos, suelos e incluso el aire, siendo ingeridos por organismos marinos y terrestres, lo que puede llevar a su acumulación en la cadena alimentaria. Su persistencia en el medio ambiente y sus potenciales efectos negativos en la salud humana y de los ecosistemas son motivo de creciente preocupación entre científicos y ambientalistas."
    )
    imagen_path = "/content/Imagenes/Microplastic.jpg"
    imagen = Image.open(imagen_path)
    return texto, imagen

def contenido_componentes_toxicos():
    texto = [
        "-Polibutileno tereftalato (PBT): Es un polímero termoplástico lineal que se fabrica para fines específicos, incluso para su uso en productos de cuidado personal (como exfoliantes, productos de baño, limpiadores faciales, pastas dentales). También se pueden utilizar en otros usos de consumo, incluidos productos de limpieza y tóneres de impresoras, y en productos industriales como medios abrasivos (por ejemplo, chorreado de plástico), industria (por ejemplo, exploración de petróleo y gas, impresión textil y moldeo automotriz), otros productos plásticos (aplicaciones antideslizantes) y aplicaciones médicas. Es probable que los productos que contienen PBT y se tiran por el desagüe se liberen en el medio acuático después del tratamiento de las aguas residuales. Los estudios han demostrado que los microplásticos, Incluido el PBT, están presentes en el medio ambiente y que pueden residir en el medio ambiente durante mucho tiempo. Se ha demostrado que el PBT provoca efectos a corto y largo plazo en organismos de laboratorio, y además este compuesto es capaz de absorber contaminantes del medio ambiente y liberarlos en el organismo que consuma o absorba el microplástico. El PBT es fácilmente absorbido por una variedad de organismos, incluidos peces, mejillones y varios tipos de zooplancton, los posibles efectos negativos de la absorción o ingesta de estos compuestos por parte de animales afectan al hígado por el agotamiento del glucógeno y la vacuolización de grasas, se encontró una regulación negativa significativa de la expresión génica de la coriogenina en los machos y una regulación negativa significativa de la vitelogenina, la coriogenina y el receptor de estrógeno alfa en las hembras.",
        "-El bisfenol A (BPA): es una sustancia química sintética que se utiliza como monómero para producir ciertos plásticos de policarbonato y resinas epoxi. Los plásticos de policarbonato y las resinas epoxi se utilizan ampliamente en productos de consumo. Algunos ejemplos son los envases de alimentos y bebidas, los equipos deportivos y de seguridad, la electrónica, las piezas de automóviles y los dispositivos médicos. El BPA también se utiliza en papel térmico recubrimientos para recibos, etiquetas de recetas y billetes de avión. El BPA se absorbe con facilidad y se metaboliza ampliamente en el organismo. Este compuesto y sus metabolitos se miden habitualmente en orina como BPA total. Estas mediciones reflejan la exposición reciente al compuesto. Los riesgos potenciales para la salud humana derivados de la exposición al BPA incluyen efectos sobre el hígado y los riñones. También incluyen efectos potenciales sobre la reproducción, el desarrollo, el neurodesarrollo y el comportamiento.",
        "-Los ftalatos: También conocidos como ésteres de ácido ftálico, son un conjunto de más de 80 compuestos químicos sintéticos utilizados en numerosas industrias como plastificadores, y cuya función principal es la de dar mayor flexibilidad y elasticidad a los polímeros rígidos. Los estudios con animales han demostrado que la exposición al DEHP puede tener efectos sobre el desarrollo y la reproducción. También puede afectar al hígado y los riñones. El Centro Internacional de Investigaciones sobre el cáncer ha clasificado el DEHP como posiblemente cancerígeno para los seres humanos. , Están presentes en productos de cuidado personal como cosméticos, también en productos de construcción y renovación como lubricantes y grasas, pinturas, revestimientos, adhesivos, selladores, textiles. Puede encontrarse en artículos eléctricos y electrónicos, también en juguetes y en algunos materiales de envasado de alimentos y plásticos de tipo PVC.",
        "-Los éteres difenílicos polibromados (PBDE): son sustancias producidas comercialmente que se utilizan como retardantes de llama para frenar la ignición y propagación del fuego en una amplia variedad de productos de consumo como las bases de alfombras, espuma para muebles, electrodomésticos y los equipos eléctricos y electrónicos. Los PBDE pueden encontrarse en el medio ambiente en todo el mundo. Los efectos críticos o importantes considerados en la evaluación de la salud humana eran efectos sobre el desarrollo neurológico (como cambios en el movimiento y el comportamiento)."
    ]
    imagen_path = "/content/Imagenes/P061337-98078.jpg"
    imagen = Image.open(imagen_path)
    return texto, imagen

# Contenido para proyecto
def contenido_objetivos_del_proyecto():
    texto = [
        "- Proporcionar una herramienta de uso libre para incentivar la colaboración ciudadana en la identificación de los microplásticos presentes en las playas del mundo.",
        "- Difundir conocimientos y recomendaciones de acción a los usuarios de esta herramienta basándose en los resultados obtenidos de la identificación.",
        "- Sentar las bases para la creación de una base de datos abiertos y así aportar al desarrollo de nuevos estudios y análisis siguiendo los Objetivos de Desarrollo Sostenible de la ONU y contribuyendo con los retos 1 y 9 de la Década Oceánica.",
    ]
    imagen_path_1 = "/content/Imagenes/S_SDG_Icons-01-06.jpg"
    imagen_1 = Image.open(imagen_path_1)
    imagen_path_2 = "/content/Imagenes/S_SDG_Icons-01-14.jpg"
    imagen_2 = Image.open(imagen_path_2)
    imagen_path_3 = "/content/Imagenes/S_SDG_Icons-01-17.jpg"
    imagen_3 = Image.open(imagen_path_3)
    imagen_path_4 = "/content/Imagenes/Understand-and-beat-marine-pollution.webp"
    imagen_4 = Image.open(imagen_path_4)
    imagen_path_5 = "/content/Imagenes/Skills-knowledge-and-technology-for-all.webp"
    imagen_5 = Image.open(imagen_path_5)
    return texto, imagen_1, imagen_2, imagen_3, imagen_4, imagen_5

# Contenido para protocolo de estimación de microplásticos en la arena
def contenido_para_protocolo_de_estimación_de_microplásticos_en_la_arena():
    texto = [
        "Para la estimación de microplásticos en la arena, se debe seguir el siguiente protocolo:",
        "1. El muestreo debe realizarse utilizando ropa de algodón, durante la marea baja y aproximadamente en el medio de la zona intermareal siguiendo una línea paralela a la costa.",
        "2. Dentro de las capacidades operativas se debe fijar cada 10 m de distancia, siguiendo la línea paralela a la costa, al menos 3 marcos de madera o metal de 50x50 cm, y con subdivisiones hechas con cuerda no sintética de 10 cm por lado.",
        "3. Se tomarán muestras de arena a 5 cm de profundidad con una cuchara metálica en las 5 subdivisiones de la diagonal del marco, las cuales serán almacenadas en frascos de vidrio con tapa metálica. \n Así, cada muestra, representará un volumen de 0.0005 m3 y servirá para aproximar el contenido de microplásticos para un volumen de 0.0125 m3, que representa el volumen de todo el marco para 5 cm de profundidad.",
    ]
    imagen_path = "/content/Imagenes/Muestreo.png"
    imagen = Image.open(imagen_path)
    return texto, imagen

def contenido_para_protocolo_de_estimación_de_microplásticos_en_la_arena2():
    texto = [
        "4. Para lograr hacer esta estimación se sugiere hacer un convenio con laboratorios de universidades o ONGs para realizar el análisis de las muestras.",
        "5. Esta iniciativa puede ser realizada por grupos de voluntarios locales o no, con interés en la conservación de las playas y la vida marina.",
    ]
    return texto

# Indicaciones
def indicaciones():
    texto = [
        "A continuación, se presentan las instrucciones para subir una imagen y obtener los resultados del análisis:",
        '1. Ten a la mano tu ubicación. Puedes obtener tus coordenadas GPS en <a href="https://www.coordenadas-gps.com/" target="_blank">https://www.coordenadas-gps.com/</a>',
        "2. Las fotos deben ser de microplásticos filtrados, es decir, sin ningún otro material en lo posible.",
        "3. Las fotos deben ser claras para obtener mejores resultados."
    ]
    return texto

##FILTROS DE COLOR

In [13]:
# Definir los rangos de color en HSV
color_ranges = {
            "Verde": ((0, 0, 200), (180, 30, 255)),
            "Azul": ((100, 150, 0), (140, 255, 255)),
            "Amarillo": ((20, 100, 100), (30, 255, 255)),
            "Blanco": ((40, 40, 40), (80, 255, 255)),
            "Rojo": ((0, 100, 100), (10, 255, 255)),
        }

##INTERFAZ

In [17]:
def crear_interfaz():

    def get_closest_color(hsv_image, x, y, w, h):
        roi = hsv_image[y:y+h, x:x+w]
        mean_color = cv2.mean(roi)[:3]
        min_dist = float('inf')
        closest_color = None
        for color, (lower, upper) in color_ranges.items():
            lower_bound = np.array(lower, dtype="uint8")
            upper_bound = np.array(upper, dtype="uint8")
            mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
            mask_mean_color = cv2.mean(hsv_image, mask=mask)[:3]
            dist = np.linalg.norm(np.array(mean_color) - np.array(mask_mean_color))
            if dist < min_dist:
                min_dist = dist
                closest_color = color
        return closest_color

    def procesar_datos(latitud, longitud, descripcion, imagen):
        imagen_np = np.array(imagen)
        outputs = predictor(imagen_np)
        instances = outputs["instances"].to("cpu")
        pred_boxes = instances.pred_boxes.tensor.numpy()  # Coordenadas de las cajas delimitadoras

        # Convertir la imagen a HSV
        hsv_image = cv2.cvtColor(imagen_np, cv2.COLOR_BGR2HSV)


        # Contar los microplásticos por color
        color_counts = {color: 0 for color in color_ranges}

        # Contar y asignar colores a cada microplástico detectado
        microplastico_data = []
        for box in pred_boxes:
            x1, y1, x2, y2 = box
            x, y, w, h = int(x1), int(y1), int(x2 - x1), int(y2 - y1)
            color = get_closest_color(hsv_image, x, y, w, h)
            if color:
                color_counts[color] += 1

                # Dibujar la caja delimitadora y el color en la imagen
                cv2.rectangle(imagen_np, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(imagen_np, color, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Agregar los datos al microplastico_data
                microplastico_data.append([color, x, y, w, h])

        # Contar los microplásticos por color y exportar a CSV
        data = {
            "Color": [],
            "Conteo": []
        }

        total_microplasticos = 0
        for color, count in color_counts.items():
            if count > 0:
                data["Color"].append(color)
                data["Conteo"].append(count)
                total_microplasticos += count

        # Agregar el total de microplásticos
        data["Color"].append("Total")
        data["Conteo"].append(total_microplasticos)

        # Convertir a DataFrame y exportar a CSV
        df_microplastico = pd.DataFrame(data)

        # Crear un DataFrame vacío para la ubicación si no hay datos
        ubicacion_data = [[latitud, longitud, descripcion]]
        df_ubicaciones = pd.DataFrame(ubicacion_data, columns=["Latitud", "Longitud", "Descripción"])

        # Concatenar ambos DataFrames
        df_final = pd.concat([df_microplastico, df_ubicaciones], axis=1)

        archivo_csv = "Momomer Analisis.csv"
        df_final.to_csv(archivo_csv, index=False)

        resumen = "Resumen del conteo de microplásticos:\n"
        for color, count in color_counts.items():
            resumen += f"{color}: {count}\n"
        resumen += f"\nTotal de microplásticos detectados: {total_microplasticos}"

        # Crear gráfico de barras
        fig, ax = plt.subplots()
        colors = list(color_counts.keys())
        counts = list(color_counts.values())
        ax.bar(colors, counts, color='skyblue')
        ax.set_xlabel('Color')
        ax.set_ylabel('Conteo')
        ax.set_title('Grafico de conteo por color')

        # Guardar gráfico como imagen
        grafico_path = "grafico_microplasticos.png"
        plt.savefig(grafico_path)
        plt.close(fig)

        # Convertir el gráfico a base64
        with open(grafico_path, "rb") as img_file:
            grafico_base64 = base64.b64encode(img_file.read()).decode('utf-8')
        grafico_html = f'<img src="data:image/png;base64,{grafico_base64}" width="400" height="300"/>'

        # Guardar la imagen con los rectángulos dibujados
        output_image_path = "Microplasticos con marcadores.jpg"
        cv2.imwrite(output_image_path, imagen_np)
        imagen_base64 = ""
        with open(output_image_path, "rb") as img_file:
            imagen_base64 = base64.b64encode(img_file.read()).decode('utf-8')
        imagen_html = f'<img src="data:image/jpeg;base64,{imagen_base64}" width="400" height="400"/>'

        # Crear enlace de descarga para el archivo CSV
        with open(archivo_csv, "rb") as file:
            csv_base64 = base64.b64encode(file.read()).decode('utf-8')
        csv_link = f'<a href="data:text/csv;base64,{csv_base64}" download="{archivo_csv}">Descargar archivo CSV</a>'

        resultados = f"Los resultados se han guardado en '{archivo_csv}'."

        return imagen_html, grafico_html, resumen, csv_link


    with gr.Blocks() as interfaz:
        # Color de la página
        gr.HTML(
        """
        <style>
            body {
                background-color: #756AB6;
            }
        </style>
        """
        )
        gr.Image(value="/content/Imagenes/Microplastics_in_the_natural_environment.jpg", label="Banner", type="pil")
        gr.Markdown(
        """
        <div style="text-align: center; font-weight: bold; font-size: 36px;">
           MonomerFinder: Identificación de microplásticos
        </div>
        """
        )

        # Contenidos desplegables 1-Definiciones
        with gr.Row():
            with gr.Column():
                with gr.Accordion("¿Qué es MonomerFinder?", open=False):
                    texto, imagen = contenido_mononerfinder()
                    gr.Image(imagen, type="pil", width=500)
                    gr.Markdown(f"<div style='padding: 5px'>{texto}</div>")

                with gr.Accordion("Fuentes de microplásticos", open=False):
                    texto, imagen = contenido_fuentes_microplasticos()
                    gr.Image(imagen, type="pil", width=500)
                    for t in texto:
                        gr.Markdown(f"<div style='padding: 5px'>{t}</div>")

            with gr.Column():
                with gr.Accordion("¿Qué son los microplásticos?", open=False):
                    texto, imagen = contenido_microplasticos()
                    gr.Image(imagen, type="pil", width=500)
                    gr.Markdown(f"<div style='padding: 5px'>{texto}</div>")

                with gr.Accordion("Componentes tóxicos presentes en los plásticos", open=False):
                    texto, imagen = contenido_componentes_toxicos()
                    gr.Image(imagen, type="pil", width=500)
                    for t in texto:
                        gr.Markdown(f"<div style='padding: 5px'>{t}</div>")

        # Contenidos desplegables 2-Proyecto
        with gr.Accordion("Objetivos del proyecto", open=False):
            texto, imagen_1, imagen_2, imagen_3, imagen_4, imagen_5 = contenido_objetivos_del_proyecto()
            for t in texto:
                gr.Markdown(f"<div style='padding: 5px'>{t}</div>")
            with gr.Row():
                gr.Image(imagen_1, type="pil", width=500)
                gr.Image(imagen_2, type="pil", width=500)
                gr.Image(imagen_3, type="pil", width=500)
            with gr.Row():
                gr.Image(imagen_4, type="pil", width=500)
                gr.Image(imagen_5, type="pil", width=500)

        # Contenidos desplegables 3-Protocolo de estimación de microplásticos en la arena
        with gr.Accordion("Protocolo de estimación de microplásticos en la arena", open=False):
            texto, imagen = contenido_para_protocolo_de_estimación_de_microplásticos_en_la_arena()
            for t in texto:
                gr.Markdown(f"<div style='padding: 5px'>{t}</div>")
            gr.Image(imagen, type="pil", width=500)
            texto = contenido_para_protocolo_de_estimación_de_microplásticos_en_la_arena2()
            for t in texto:
                gr.Markdown(f"<div style='padding: 5px'>{t}</div>")

        #Ingreso de imagen
        gr.Markdown(
        """
        <div style="text-align: center; font-weight: bold; font-size: 36px;">
           ¡Vamos a identificar y contar microplásticos!
        </div>
        """
        )
        texto = indicaciones()
        for t in texto:
            gr.Markdown(f"<div style='padding: 5px'>{t}</div>")
        gr.Markdown(
        """
        <div style="text-align: center; font-weight: bold; font-size: 30px;">
           ¡Sube tu imagen!
        </div>
        """
        )

        #Ingreso ubicación
        latitud = gr.Textbox(label="Latitud")
        longitud = gr.Textbox(label="Longitud")
        descripcion = gr.Textbox(label="Descripción")
        imagen = gr.Image(label="Cargar Imagen", type="pil")

        boton_procesar = gr.Button("Procesar Imagen")
        salida_imagen = gr.HTML()
        salida_grafico = gr.HTML()
        salida_resumen = gr.HTML()
        salida_csv = gr.HTML()

        boton_procesar.click(fn=procesar_datos, inputs=[latitud, longitud, descripcion, imagen], outputs=[salida_imagen, salida_grafico, salida_resumen, salida_csv])

    return interfaz

# Crear y lanzar la interfaz
interfaz = crear_interfaz()
interfaz.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7bec85a083c458e469.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
